# Квантизация

Квантизованную модель нельзя обучать, и она работает медленнее, но зато занимает меньше места в памяти GPU.

[A Gentle Introduction to 8-bit Matrix Multiplication for transformers at scale using Hugging Face Transformers, Accelerate and bitsandbytes](https://huggingface.co/blog/hf-bitsandbytes-integration)

[Making LLMs even more accessible with bitsandbytes, 4-bit quantization and QLoRA](https://huggingface.co/blog/4bit-transformers-bitsandbytes)

[Quantize 🤗 Transformers models](https://huggingface.co/docs/transformers/main_classes/quantization)

![image](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/blog/bitsandbytes/FP8-scheme.png)

In [1]:
%pip --quiet install transformers bitsandbytes accelerate

Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import AutoModelForCausalLM
import torch

model_name = "ai-forever/ruGPT-3.5-13B"
model = AutoModelForCausalLM.from_pretrained(model_name, load_in_8bit=True, torch_dtype=torch.float16)

model_size = sum(t.numel() for t in model.parameters())
print(f"model_size: {model_size/1000**2:.1f}M")

model

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

model_size: 12853.5M


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50272, 5120)
    (wpe): Embedding(2048, 5120)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-39): 40 x GPT2Block(
        (ln_1): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Linear8bitLt(in_features=5120, out_features=15360, bias=True)
          (c_proj): Linear8bitLt(in_features=5120, out_features=5120, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Linear8bitLt(in_features=5120, out_features=20480, bias=True)
          (c_proj): Linear8bitLt(in_features=20480, out_features=5120, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((5120,), eps=1e-

In [3]:
from transformers import AutoTokenizer, GenerationConfig

tokenizer = AutoTokenizer.from_pretrained(model_name)

print(tokenizer.special_tokens_map)

{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<|endoftext|>', 'pad_token': '<pad>', 'mask_token': '<mask>'}


In [4]:
generation_config = GenerationConfig(
  bos_token_id=tokenizer.bos_token_id,
  eos_token_id=tokenizer.eos_token_id,
  pad_token_id=tokenizer.pad_token_id,
  do_sample=True,
  max_new_tokens=1536,
  no_repeat_ngram_size=15,
  repetition_penalty=1.15,
  temperature=0.2,
  top_k=30,
  top_p=0.9,
)

generation_config

GenerationConfig {
  "bos_token_id": 2,
  "do_sample": true,
  "eos_token_id": 3,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.15,
  "temperature": 0.2,
  "top_k": 30,
  "top_p": 0.9
}

In [5]:
prefix = 'О сколько нам открытий чудных'

inputs = tokenizer(prefix, return_tensors='pt').to('cuda')

outputs = model.generate(
    **inputs, 
    generation_config=generation_config,
    )

print(tokenizer.decode(outputs[0]))

О сколько нам открытий чудных...
Готовит просвещенья дух,
И опыт сын ошибок трудных,
И гений парадоксов друг.<|endoftext|>
Вдохновение: Дима Днк "Я не хочу быть один" http://www.stihi.ru/2017/10/20/7995

Я не хочу быть один,
Хочу с тобою рядом жить!
С тобой я буду счастлив,
Ведь ты моя награда!

Ты моё счастье и мечта,
Моя любовь и красота!
Ты мой рассвет и закат,
Мой нежный ангелок!

Твои глаза как два огня,
Они манят меня в даль.
Как же мне хочется обнять тебя,
Но это лишь мечты.

Мне очень жаль что мы сейчас не вместе,
Но знай,что я всегда с тобой.
Мы будем счастливы поверь мне,
Ведь я люблю тебя душой!!!<|endoftext|>
Не надо слов - они пусты, 
Когда душа твоя молчит. 
Словами чувства не спасти, 
А без любви на сердце стынет лед. 

Не говори, что любишь ты, 
Что хочешь видеть каждый день. 
Любовь бывает так слепа, 
Она приходит только раз. 

Не говори о том, что ждешь, 
Что будешь ждать всю жизнь свою. 
Любви ведь тоже есть предел, 
Его ты знаешь по себе. 

Не обещай того, чего 
Сам